In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

import datetime
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math


In [2]:
df = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [3]:
df.start_at=pd.to_datetime(df.start_at)


In [4]:
df["drama_key"] = df.start_at.dt.strftime("%y")+df.start_at.dt.strftime("%m")+"_"+df.TV_station+"_"+df.time_table

In [5]:
df
# df.start_at.dt.strftime("%m")

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,8.8,9.8,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,9.7,-,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,9.6,-,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,13.7,-,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,19.3,-,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,3.2,-,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,9.2,-,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,10.4,-,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22


In [6]:
# import pandas_profiling as pdp
# pdp.ProfileReport(df)

In [7]:
import json

f = open("../data/drama_info.json", 'r')
print(f)
drama_info_json = json.load(f)

<_io.TextIOWrapper name='../data/drama_info.json' mode='r' encoding='UTF-8'>


In [8]:
# def addDramaInfo (drama_title) 
#     main_actor_only_json[drama_title]
    


In [9]:
print (drama_info_json)

{'1910_CX_月21': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_水22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_木23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_金22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_金23': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_土22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_土23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_日21': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_日22': {'main_actor1': '', 'main_actor2': ''}, '1907_CX_月21': {'main_actor1': '上野樹里', 'main_actor2': '時任三郎'}, '1907_CX_火21': {'main_actor1': '三浦春馬', 'main_actor2': '芳根京子'}, '1907_TBS_火22': {'main_actor1': '石原さとみ', 'main_actor2': '福士蒼汰'}, '1907_NTV_水22': {'main_actor1': '杏', 'main_actor2': '宮沢氷魚'}, '1907_EX_木21': {'main_act

In [10]:
df["main_actor1"] = ""
df["main_actor2"] = ""

In [11]:
for tmp_index in range(len(df)):
    tmp_drama_key = df.iloc[tmp_index]["drama_key"]
    if tmp_drama_key in drama_info_json.keys():
#         print(drama_info_json[tmp_drama_key]["main_actor1"])
        df["main_actor1"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor1"]
        df["main_actor2"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor2"]
#         print("yes"+tmp_drama_title)
    else:
        print(tmp_drama_key)

In [12]:
df

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21,,
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22,,
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22,,
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22,,
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22,,
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22,,
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23,,
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22,,
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23,,
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22,,


In [13]:
def make_person_key_df (actor_actress):
    actor_actress_list = []
    for column in actor_actress.columns:
        if column == "ranking":
            continue
        actor_actress_list.extend(actor_actress[column].values.tolist())
    print ("元の数"+ str(len(actor_actress_list)))
    # uniqueにする
    actor_actress_list = list(set(actor_actress_list))
    #nan を除去
    actor_actress_list = [actor for actor in actor_actress_list if type(actor) is not float ]
    print ("後の数"+str(len(actor_actress_list)))

    df_pivot = pd.DataFrame( columns=actor_actress_list, index=[str(n) for n in range(2008,2020)])

    #pivot 形式に整形する
    for tmp_name in df_pivot.columns:
        for year in actor_actress.columns:
            if year == "ranking": continue
            if  len(actor_actress[actor_actress[year] == tmp_name].ranking) > 0:
                df_pivot.loc[year][tmp_name] = (actor_actress[actor_actress[year] == tmp_name].ranking).values[0]
    return (df_pivot)
    

In [14]:
df_actress = pd.read_csv("../data/actress_power.csv", encoding='cp932')
df_actress_pivot = make_person_key_df(df_actress)


元の数360
後の数62


In [15]:
df_actor = pd.read_csv("../data/actor_power.csv", encoding='cp932')
df_actor_pivot = make_person_key_df(df_actor)

元の数372
後の数65


In [16]:
df_actress_pivot

,深田恭子,石原さとみ,綾瀬はるか,菅野美穂,多部未華子,小泉今日子,真矢みき,木村佳乃,広瀬アリス,戸田恵梨香,...,有村架純,常盤貴子,桐谷美玲,米倉涼子,芦田愛菜,蒼井優,上野樹里,樹木希林,新垣結衣,波瑠
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,20,NaN,6,5,NaN,NaN,18,NaN,NaN,25,...,NaN,24,NaN,19,NaN,26,NaN,NaN,9,NaN
2011,17,21,2,1,NaN,16,20,NaN,NaN,NaN,...,NaN,NaN,NaN,26,NaN,25,18,NaN,10,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,18,8,1,6,NaN,11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13,NaN,NaN,NaN,24,5,NaN
2015,18,12,1,15,NaN,24,NaN,NaN,NaN,NaN,...,NaN,NaN,29,9,NaN,NaN,30,NaN,5,NaN
2016,8,5,1,18,NaN,19,NaN,NaN,NaN,NaN,...,3,NaN,10,20,NaN,NaN,NaN,NaN,4,NaN
2017,5,3,2,17,24,27,NaN,NaN,NaN,NaN,...,6,NaN,9,18,NaN,NaN,NaN,NaN,1,NaN


In [17]:
df_actor_namekey = df_actor_pivot.T.rename_axis('actor_actress').reset_index()

In [18]:
df_actress_namekey = df_actress_pivot.T.rename_axis('actor_actress').reset_index()

In [19]:
df_actress_namekey

,actor_actress,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,深田恭子,NaN,NaN,20,17,NaN,NaN,18,18,8,5,4,4
1,石原さとみ,NaN,NaN,NaN,21,NaN,NaN,8,12,5,3,3,5
2,綾瀬はるか,NaN,NaN,6,2,NaN,NaN,1,1,1,2,1,1
3,菅野美穂,NaN,NaN,5,1,NaN,NaN,6,15,18,17,19,28
4,多部未華子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,22,14
5,小泉今日子,NaN,NaN,NaN,16,NaN,NaN,11,24,19,27,NaN,NaN
6,真矢みき,NaN,NaN,18,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,木村佳乃,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30
8,広瀬アリス,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,22
9,戸田恵梨香,NaN,NaN,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,11


In [20]:
df_merged = pd.merge(df, df_actor_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
df_merged = pd.merge(df_merged, df_actor_namekey,left_on='main_actor2', right_on='actor_actress', how='right')
df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='right')

In [21]:
# df_merged[["main_actor1","2017","2016","2014"]]
df_merged.columns

Index(['TV_station', 'time_table', 'drama_title', 'Episode1', 'Episode2',
       'Episode3', 'Episode4', 'Episode5', 'Episode6', 'Episode7', 'Episode8',
       'Episode9', 'Episode10', 'Episode11', 'Episode12', 'Episode13',
       'Episode14', 'Episode15', 'average', 'suii', 'start_at', 'drama_key',
       'main_actor1', 'main_actor2', 'actor_actress_x', '2008_x', '2009_x',
       '2010_x', '2011_x', '2012_x', '2013_x', '2014_x', '2015_x', '2016_x',
       '2017_x', '2018_x', '2019_x', 'actor_actress_y', '2008_y', '2009_y',
       '2010_y', '2011_y', '2012_y', '2013_y', '2014_y', '2015_y', '2016_y',
       '2017_y', '2018_y', '2019_y', 'actor_actress_x', '2008_x', '2009_x',
       '2010_x', '2011_x', '2012_x', '2013_x', '2014_x', '2015_x', '2016_x',
       '2017_x', '2018_x', '2019_x', 'actor_actress_y', '2008_y', '2009_y',
       '2010_y', '2011_y', '2012_y', '2013_y', '2014_y', '2015_y', '2016_y',
       '2017_y', '2018_y', '2019_y'],
      dtype='object')